## AIHub Json Preprocessing

### Development Environment

In [ ]:
%pip install kss==3.7.3

KSS Argument Error: Restart Jupyter Kernel Runtime

In [ ]:
%pip install python-mecab-ko

KSS 3.7.3 matches python-mecab-ko

In [ ]:
%pip install pandas

In [ ]:
%pip install ray

In [1]:
import re
import os
import kss
import ray
import json
import time
from time import sleep
from tqdm import tqdm
from mecab import MeCab
import pandas as pd
from glob import glob
from itertools import chain

In [2]:
pwd

'D:\\AIHUB'

### Basic Function

In [2]:
def sorted_list(path_list):
    
    path_list = sorted(path_list, reverse=False)
    path_list = sorted(path_list, key=len)
    
    return path_list

In [3]:
def divide_source_file_list(l, n): 
    
  for i in range(0, len(l), n): 
    yield l[i:i + n] 

In [4]:
def json_file_path_list(path_list):
    
    file_path  = [glob(i, recursive = True) for i in path_list][0]
    file_path = sorted_list(file_path)
   
    return file_path

In [5]:
def train_valid_json_file_path_list(path_list):

  train_file_path, valid_file_path = [glob(i, recursive = True) if 'rain' in i
                                      else glob(i, recursive = True)
                                      for i in path_list]

  train_file_path = sorted_list(train_file_path)
  valid_file_path = sorted_list(valid_file_path)

  return train_file_path, valid_file_path

In [6]:
def txt_file_path_list(source_file_nested_list, folder_corpus_type_path):

  text_file_path_list = [folder_corpus_type_path + str(i) + ".txt"
                              for i in range(len(source_file_nested_list))]
    
  return text_file_path_list

In [7]:
def make_train_valid_json_txt_file_path_list(json_path_list, txt_path_list):

    train_file_path, valid_file_path = train_valid_json_file_path_list(json_path_list)
    
    the_number_of_json_file = len(train_file_path) + len(valid_file_path)
    print("The number of file:", the_number_of_json_file)
    
    train_text_file_path_list = txt_file_path_list(train_file_path, txt_path_list[0])
    valid_text_file_path_list = txt_file_path_list(valid_file_path, txt_path_list[1])

    return train_file_path, valid_file_path, train_text_file_path_list, valid_text_file_path_list

In [8]:
def make_json_txt_file_path_list(json_path_list, txt_path_list):

    file_path = json_file_path_list(json_path_list)
    
    the_number_of_json_file = len(file_path) 
    print("The number of file:", the_number_of_json_file)
    
    text_file_path_list = txt_file_path_list(file_path, txt_path_list[0])

    return file_path, text_file_path_list

In [9]:
def legal_preprocessing_text(source):
    preprocessing_sentence_list = []
    
    source = source.strip()
    # strip으로 앞뒤 공백 제거

    source = re.sub(r"\[.*?\]|\{.*?\}", "", source)
    # 기타 괄호 제거할 시 괄호 내부에 모든 텍스트 제거


    try:
        bracket_form = re.compile('\(([^)]+)')
        text_in_small_bracket = bracket_form.findall(source)
    
    
        if type(text_in_small_bracket) == str:

            text = text_in_small_bracket

            text_size = len(text)
            last_index = source.find(text) + len(text)
            if len(source) >= last_index+1 and source[last_index-text_size-1] == '(' and source[last_index+1] == '.':
                source = source.replace(source[last_index-text_size-1 : last_index+1] + ".", ".")

            if len(text.split()) > 5 and bool(re.match(r'[.]|[!]|[?]', text[-1])) == True:
                small_bracket = "(" + text + ")"
                source = source.replace(small_bracket, " " + text + " ")    

        elif type(text_in_small_bracket) == list:

            for text in text_in_small_bracket:

                text_size = len(text)
                last_index = source.find(text) + len(text)
                if len(source) >= last_index+1 and source[last_index-text_size-1] == '(' and source[last_index+1] == '.':
                    source = source.replace(source[last_index-text_size-1 : last_index+1] + ".", ".")

                if len(text.split()) > 5 and bool(re.match(r'[.]|[!]|[?]', text[-1])) == True:
                    small_bracket = "(" + text + ")"
                    source = source.replace(small_bracket, " " + text + " ")    

    except:
        pass

        # 마침표(.) 앞에 소괄호')'가 있을시 소괄호 제거와 함께 소괄호 내부 텍스트 제거
        # 소괄호 내부 텍스트가 5어절 이상이고 끝이 온점(.). 느낌표(!). 물음표(?)일 떼 소괄호 제거
        
    
    if bool(re.match(r'[가나다라마바사아자차카타파하]+[.]', source[:2])) == True:
        source = source.replace(source[:2], "")
        
    source = re.sub(r' [가나다라마바사아자차카타파하]+[.]', "", source)
    # '가.', '나.', ... 형태의 문자열 제거   
    
    sources = re.split(r"[0-9]+[.]|①|②|③|④|⑤|⑥|⑦|⑧|⑨|⑩|⑪|⑫|⑬|⑭|⑮", source)
    # 숫자.(1. 2. ...), 원숫자(①, ②, ③, ...)를 기준으로 문장 분리

    for source in sources:
        
        for sentence in kss.split_sentences(source, use_heuristic=False,
                                            num_workers=32):
        # KSS(Korean Sentence Segmentation)로 문장 분리 
        # Formal articles (wiki, news, essays): recommend to False
        
            if re.search("^[A-Za-z0-9ㄱ-ㅎ가-힣一-鿕㐀-䶵豈-龎]", sentence[0]) is not None and \
                bool(re.match(r'[.]|[!]|[?]', sentence[-1])) == True and \
                len(sentence.split()) > 5:
                # 문장의 시작이 특수문자인 문장(영어 대소문자, 한글, 한자, 숫자, -, + 제외
                # 문장의 끝이 온점(.). 느낌표(!). 물음표(?)가 아닌 문장 제외
                # 다섯 어절 이하 문장 제외


                if ']' in sentence and '[' not in sentence:
                    sentence  = re.sub(r".*?]", "", sentence)    
                # 중괄호 앞에 있는 '성명/직함]' 형태 제거


                sentence = re.sub(r"[^A-Za-z0-9ㄱ-ㅎ가-힣一-鿕㐀-䶵豈-龎()+-.,]", " ", sentence)
                # 특수문자 제거(영어 대소문자, 한글, 한자, 숫자, -, +, 소괄호, 마침표, 쉼표, 제외)

                sentence = sentence.strip()
                # strip으로 앞뒤 공백 제거

                total_length = len(sentence.replace(" " , ""))
                hangeul_length = len(re.sub(r"[^ㄱ-ㅣ가-힣\s]", "", sentence.replace(" " , "")))
                hangeul_ratio = hangeul_length / total_length
                if hangeul_ratio >= 0.5:
                # 한글이 아닌 문자열이 50% 이상이 넘은 문장 제외
                    preprocessing_sentence_list.append(sentence)

    return preprocessing_sentence_list

### AIHUB 법률 규정 (판결서 약관 등) 텍스트 분석 데이터

[Source](https://aihub.or.kr/aihubdata/data/view.do?currMenu=116&topMenu=100&aihubDataSe=ty&dataSetSn=580)

#### Convert JSON File to TXT File

In [10]:
json_path_list = ['AIHUB_법률 규정 (판결서 약관 등) 텍스트 분석 데이터/Training' + '/**/*.json',
             'AIHUB_법률 규정 (판결서 약관 등) 텍스트 분석 데이터/Validation' + '/**/*.json']
txt_path_list = ["exploration/legal_regulations_(such_as_terms_and_conditions_of_judgment)_text_analysis_data_pro/AIHUB_legal_regulations_(such_as_terms_and_conditions_of_judgment)_text_analysis_data_train_", 
                 "exploration/legal_regulations_(such_as_terms_and_conditions_of_judgment)_text_analysis_data_pro/AIHUB_legal_regulations_(such_as_terms_and_conditions_of_judgment)_text_analysis_data_valid_"]

In [11]:
train_file_list, valid_file_list, train_text_file_path_list, valid_text_file_path_list = \
    make_train_valid_json_txt_file_path_list(json_path_list, txt_path_list)

The number of file: 9450


In [12]:
source_file_index_df = pd.DataFrame(train_file_list, columns=['source_file_name'])
source_file_index_df.to_excel("source_file_index/legal_regulations_(such_as_terms_and_conditions_of_judgment)_text_analysis_data_source_train_file_index.xlsx")

source_file_index_df = pd.DataFrame(valid_file_list, columns=['source_file_name'])
source_file_index_df.to_excel("source_file_index/legal_regulations_(such_as_terms_and_conditions_of_judgment)_text_analysis_data_source_valid_file_index.xlsx")

In [13]:
def make_sources(source_file, one_json_sample):
    
    if '판결문' in source_file:
        source_df = pd.DataFrame(one_json_sample)
        sources = source_df.loc['disposalcontent']['disposal'] + \
            source_df.loc['rqestObjet']['mentionedItems'] + \
            source_df.loc['acusrAssrs']['assrs'] + \
            source_df.loc['dedatAssrs']['assrs'] + \
            source_df.loc['bsisFacts']['facts'] + \
            source_df.loc['courtDcss']['dcss'] + \
            source_df.loc['cnclsns']['close']

    elif '유리' in source_file:
        source_df = pd.DataFrame.from_dict(one_json_sample, orient='index')
        source_df = source_df.transpose()
        sources = [source_df['clauseArticle'][0], source_df['comProvision'][0]]

    elif '불리' in source_file:
        source_df = pd.DataFrame.from_dict(one_json_sample, orient='index')
        source_df = source_df.transpose()
        sources = [source_df['clauseArticle'][0][0], source_df['illdcssBasiss'][0][0],
                        source_df['relateLaword'][0][0]]
        
    return sources

In [14]:
def count_number_of_txt_file_with_batch_list(source_file_list, batch_size):
    
    source_file_by_batch_df = pd.DataFrame({'File':[0], 'Length of Source List':[0],
                                            'The Number of txt File':[0], 
                                            'Description':[0]})
    the_number_of_total_txt_file = 0
    the_number_of_txt_file_list = []
    source_list = []
    
    for i in range(len(source_file_list)):    
        
        source_file = source_file_list[i]   
            
        with open(source_file, 'r', encoding='utf-8') as one_json_file:
            one_json_sample = json.load(one_json_file) 

        sources = make_sources(source_file, one_json_sample)
        for source in sources:
            source_list.append(source[0])
            
        the_number_of_txt_file = 1

        if len(source_list) >= batch_size:
            source_file_by_batch_df.loc[i] = [source_file,
                                              len(source_list), the_number_of_txt_file, ""]
              
        elif len(source_list) < batch_size:
            source_file_by_batch_df.loc[i] = [source_file,
                                              len(source_list), the_number_of_txt_file,
                                              "not subject of batch. small source list."]
            
        the_number_of_txt_file_list.append(len(source_list))
        the_number_of_total_txt_file += len(source_list) 
        source_list = []

    the_number_of_total_txt_file = the_number_of_total_txt_file // batch_size                 
    print("Batch Size:", batch_size)
    print("The number of txt file:", the_number_of_total_txt_file)

    source_file_by_batch_df = source_file_by_batch_df.astype({'Length of Source List':'int', 
                                                              'The Number of txt File':'int'})
    
    if 'rain' in source_file:
        source_file_by_batch_df.to_excel("source_file_by_batch/legal_regulations_(such_as_terms_and_conditions_of_judgment)_text_analysis_data_train.xlsx")
    elif 'alid' in source_file:
        source_file_by_batch_df.to_excel("source_file_by_batch/legal_regulations_(such_as_terms_and_conditions_of_judgment)_text_analysis_data_valid.xlsx")
    else:
         source_file_by_batch_df.to_excel("source_file_by_batch/legal_regulations_(such_as_terms_and_conditions_of_judgment)_text_analysis_data.xlsx")
    
    return the_number_of_total_txt_file, the_number_of_txt_file_list

In [15]:
def write_jsontext_to_txt_file_with_batch_list(source_file_list,
                                    text_file_path_list,
                                    batch_size, the_number_of_txt_file_list):

  progress_length = sum(the_number_of_txt_file_list) // batch_size
  print("[Size]")
  print("The number of preprocessing corpus: " + str(progress_length))
  print("\n[Order]")
  source_list = []
  pbar = tqdm(range(progress_length))
  num = 0
  
  for i in range(len(source_file_list)):

    source_file = source_file_list[i]
        
    with open(source_file, 'r', encoding='utf-8') as one_json_file:
      one_json_sample = json.load(one_json_file)
      
    sources = make_sources(source_file, one_json_sample)

    if len(source_list) >= batch_size:
        with open(os.path.join('AIHUB_corpus/' + text_file_path_list[i][:-4] + ".txt"), "a", encoding='utf-8') as fp:        
            fp.write("\n".join(source_list)) 
        pbar.n += 1
        pbar.refresh()
        time.sleep(0.01)  
  
        source_list = []
          
    elif i == (len(source_file_list) -1): 
        for source in sources:
          source_list.append(source)
        
        with open(os.path.join('AIHUB_corpus/' + text_file_path_list[i][:-4] + ".txt"), "a", encoding='utf-8') as fp:        
            fp.write("\n".join(source_list[0])) 
        num += 1  
        pbar.n += 1
        pbar.refresh()
        time.sleep(0.01)
                    
    for source in sources:
      source_list.append(source[0])   
  pbar.close()      

In [16]:
batch_size = 1000
the_number_of_txt_file, the_number_of_txt_file_list = count_number_of_txt_file_with_batch_list(train_file_list, batch_size)

Batch Size: 1000
The number of txt file: 30


In [17]:
source_file_by_batch_train_df = pd.read_excel('source_file_by_batch/legal_regulations_(such_as_terms_and_conditions_of_judgment)_text_analysis_data_train.xlsx', engine='openpyxl')  
source_file_by_batch_train_df

,Unnamed: 0,File,Length of Source List,The Number of txt File,Description
0,0,AIHUB_법률 규정 (판결서 약관 등) 텍스트 분석 데이터/Training\라벨링...,2,1,not subject of batch. small source list.
1,1,AIHUB_법률 규정 (판결서 약관 등) 텍스트 분석 데이터/Training\라벨링...,2,1,not subject of batch. small source list.
2,2,AIHUB_법률 규정 (판결서 약관 등) 텍스트 분석 데이터/Training\라벨링...,2,1,not subject of batch. small source list.
3,3,AIHUB_법률 규정 (판결서 약관 등) 텍스트 분석 데이터/Training\라벨링...,2,1,not subject of batch. small source list.
4,4,AIHUB_법률 규정 (판결서 약관 등) 텍스트 분석 데이터/Training\라벨링...,2,1,not subject of batch. small source list.
...,...,...,...,...,...
8395,8395,AIHUB_법률 규정 (판결서 약관 등) 텍스트 분석 데이터/Training\라벨링...,35,1,not subject of batch. small source list.
8396,8396,AIHUB_법률 규정 (판결서 약관 등) 텍스트 분석 데이터/Training\라벨링...,15,1,not subject of batch. small source list.
8397,8397,AIHUB_법률 규정 (판결서 약관 등) 텍스트 분석 데이터/Training\라벨링...,77,1,not subject of batch. small source list.
8398,8398,AIHUB_법률 규정 (판결서 약관 등) 텍스트 분석 데이터/Training\라벨링...,55,1,not subject of batch. small source list.


In [18]:
batch_size = 1000
write_jsontext_to_txt_file_with_batch_list(train_file_list, train_text_file_path_list,
                batch_size, the_number_of_txt_file_list)

[Size]
The number of preprocessing corpus: 30

[Order]


  0%|          | 0/30 [00:00<?, ?it/s]

31it [00:10,  2.90it/s]                        


In [19]:
batch_size = 1000
the_number_of_txt_file, the_number_of_txt_file_list = count_number_of_txt_file_with_batch_list(valid_file_list, batch_size)

Batch Size: 1000
The number of txt file: 4


In [164]:
source_file_by_batch_valid_df = pd.read_excel('source_file_by_batch/legal_regulations_(such_as_terms_and_conditions_of_judgment)_text_analysis_data_valid.xlsx', engine='openpyxl')  
source_file_by_batch_valid_df

,Unnamed: 0,File,Length of Source List,The Number of txt File,Description
0,0,AIHUB_법률 규정 (판결서 약관 등) 텍스트 분석 데이터/Validation\라...,2,1,not subject of batch. small source list.
1,1,AIHUB_법률 규정 (판결서 약관 등) 텍스트 분석 데이터/Validation\라...,2,1,not subject of batch. small source list.
2,2,AIHUB_법률 규정 (판결서 약관 등) 텍스트 분석 데이터/Validation\라...,2,1,not subject of batch. small source list.
3,3,AIHUB_법률 규정 (판결서 약관 등) 텍스트 분석 데이터/Validation\라...,2,1,not subject of batch. small source list.
4,4,AIHUB_법률 규정 (판결서 약관 등) 텍스트 분석 데이터/Validation\라...,2,1,not subject of batch. small source list.
...,...,...,...,...,...
1045,1045,AIHUB_법률 규정 (판결서 약관 등) 텍스트 분석 데이터/Validation\라...,56,1,not subject of batch. small source list.
1046,1046,AIHUB_법률 규정 (판결서 약관 등) 텍스트 분석 데이터/Validation\라...,36,1,not subject of batch. small source list.
1047,1047,AIHUB_법률 규정 (판결서 약관 등) 텍스트 분석 데이터/Validation\라...,28,1,not subject of batch. small source list.
1048,1048,AIHUB_법률 규정 (판결서 약관 등) 텍스트 분석 데이터/Validation\라...,30,1,not subject of batch. small source list.


In [134]:
batch_size = 1000
write_jsontext_to_txt_file_with_batch_list(valid_file_list, valid_text_file_path_list,
                batch_size, the_number_of_txt_file_list)

[Size]
The number of preprocessing corpus: 4

[Order]


100%|██████████| 4/4 [00:01<00:00,  3.37it/s]


#### Preprocess TXT File

In [136]:
def post_txt_file_path_list(corpus_path_list):
   
  post_corpus_path_list = [corpus_file.replace("pro", "post")
                      for corpus_file in corpus_path_list]

  return post_corpus_path_list

In [137]:
def make_pro_post_txt_file_path_list(pro_corpus_path):
    
    pro_total_corpus_path_list = glob(pro_corpus_path)
    pro_total_corpus_path_list = sorted_list(pro_total_corpus_path_list)
    post_total_corpus_path_list = post_txt_file_path_list(pro_total_corpus_path_list)

    return pro_total_corpus_path_list, post_total_corpus_path_list

In [138]:
pro_corpus_path = "AIHUB_corpus/exploration/legal_regulations_(such_as_terms_and_conditions_of_judgment)_text_analysis_data_pro/AIHUB_legal_regulations_(such_as_terms_and_conditions_of_judgment)_text_analysis_data_" + "*.txt"
pro_total_corpus_path_list, post_total_corpus_path_list = make_pro_post_txt_file_path_list(pro_corpus_path)

In [139]:
len(pro_total_corpus_path_list)

35

In [141]:
line_list = []
line_num = 0
with open(pro_total_corpus_path_list[0], 'r', encoding='utf-8') as f:
    lines = f.read().splitlines() 
    for line in lines:
        line_num += 1
        if line_num <= 2:
           line_list.append(line)
for line in line_list:
    print(line, end="\n\n")

제2조(보증금액)

 ① 이 보증서에 의한 보증금액은 채권자의 채무자에 대한 보증부대출 예정금액에 보증비율을 곱한 금액으로 합니다.



In [142]:
line_list = []
line_num = 0
with open(pro_total_corpus_path_list[0], 'r', encoding='utf-8') as f:
    lines = f.read().splitlines()
    for line in lines:
        line_num += 1
        if line_num <= 2:  
            sentences = legal_preprocessing_text(line)
            for sentence in sentences:
                line_list.append(sentence) 
            
for line in line_list:
    print(line, end="\n\n")

이 보증서에 의한 보증금액은 채권자의 채무자에 대한 보증부대출 예정금액에 보증비율을 곱한 금액으로 합니다.



In [155]:
ray.init(num_cpus = 4)

@ray.remote
def legal_preprocessing_text(source):
    preprocessing_sentence_list = []
    
    source = source.strip()
    # strip으로 앞뒤 공백 제거

    source = re.sub(r"\[.*?\]|\{.*?\}", "", source)
    # 기타 괄호 제거할 시 괄호 내부에 모든 텍스트 제거

    try:
        bracket_form = re.compile('\(([^)]+)')
        text_in_small_bracket = bracket_form.findall(source)
    
    
        if type(text_in_small_bracket) == str:

            text = text_in_small_bracket

            text_size = len(text)
            last_index = source.find(text) + len(text)
            if len(source) >= last_index+1 and source[last_index-text_size-1] == '(' and source[last_index+1] == '.':
                source = source.replace(source[last_index-text_size-1 : last_index+1] + ".", ".")

            if len(text.split()) > 5 and bool(re.match(r'[.]|[!]|[?]', text[-1])) == True:
                small_bracket = "(" + text + ")"
                source = source.replace(small_bracket, " " + text + " ")    

        elif type(text_in_small_bracket) == list:

            for text in text_in_small_bracket:

                text_size = len(text)
                last_index = source.find(text) + len(text)
                if len(source) >= last_index+1 and source[last_index-text_size-1] == '(' and source[last_index+1] == '.':
                    source = source.replace(source[last_index-text_size-1 : last_index+1] + ".", ".")

                if len(text.split()) > 5 and bool(re.match(r'[.]|[!]|[?]', text[-1])) == True:
                    small_bracket = "(" + text + ")"
                    source = source.replace(small_bracket, " " + text + " ")    

    except:
        pass

        # 마침표(.) 앞에 소괄호')'가 있을시 소괄호 제거와 함께 소괄호 내부 텍스트 제거
        # 소괄호 내부 텍스트가 5어절 이상이고 끝이 온점(.). 느낌표(!). 물음표(?)일 떼 소괄호 제거
        
    
    if bool(re.match(r'[가나다라마바사아자차카타파하]+[.]', source[:2])) == True:
        source = source.replace(source[:2], "")
        
    source = re.sub(r' [가나다라마바사아자차카타파하]+[.]', "", source)
    # '가.', '나.', ... 형태의 문자열 제거  
    
    
    sources = re.split(r"[0-9]+[.]|①|②|③|④|⑤|⑥|⑦|⑧|⑨|⑩|⑪|⑫|⑬|⑭|⑮", source)
    # 숫자.(1. 2. ...), 원숫자(①, ②, ③, ...)를 기준으로 문장 분리

    for source in sources:
        
        for sentence in kss.split_sentences(source, use_heuristic=False,
                                            num_workers=32):
        # KSS(Korean Sentence Segmentation)로 문장 분리 
        # Formal articles (wiki, news, essays): recommend to False


            if re.search("^[A-Za-z0-9ㄱ-ㅎ가-힣一-鿕㐀-䶵豈-龎]", sentence[0]) is not None and \
                bool(re.match(r'[.]|[!]|[?]', sentence[-1])) == True and \
                len(sentence.split()) > 5:
                # 문장의 시작이 특수문자인 문장(영어 대소문자, 한글, 한자, 숫자, -, + 제외
                # 문장의 끝이 온점(.). 느낌표(!). 물음표(?)가 아닌 문장 제외
                # 다섯 어절 이하 문장 제외


                if ']' in sentence and '[' not in sentence:
                    sentence  = re.sub(r".*?]", "", sentence)    
                # 중괄호 앞에 있는 '성명/직함]' 형태 제거


                sentence = re.sub(r"[^A-Za-z0-9ㄱ-ㅎ가-힣一-鿕㐀-䶵豈-龎()+-.,]", " ", sentence)
                # 특수문자 제거(영어 대소문자, 한글, 한자, 숫자, -, +, 소괄호, 마침표, 쉼표, 제외)

                sentence = sentence.strip()
                # strip으로 앞뒤 공백 제거

                total_length = len(sentence.replace(" " , ""))
                hangeul_length = len(re.sub(r"[^ㄱ-ㅣ가-힣\s]", "", sentence.replace(" " , "")))
                hangeul_ratio = hangeul_length / total_length
                if hangeul_ratio >= 0.5:
                # 한글이 아닌 문자열이 50% 이상이 넘은 문장 제외
                    preprocessing_sentence_list.append(sentence)

    return preprocessing_sentence_list

2023-07-11 14:08:05,201	INFO worker.py:1625 -- Started a local Ray instance.


In [156]:
def preprocessing_corpus_txt(pro_total_corpus_path_list, post_total_corpus_path_list):

    progress_length = len(pro_total_corpus_path_list)
    print("[Size]")
    print("The number of preprocessing corpus: " + str(len(pro_total_corpus_path_list)))
    print("\n[Order]")
    pbar = tqdm(range(progress_length))
    process_num = 10    

    for pro, post in zip(pro_total_corpus_path_list, post_total_corpus_path_list):

        sentence_list = []

        with open(pro, 'r', encoding='utf-8') as f:
            lines = f.read().splitlines() 
            nested_lines_num = len(lines) // process_num
            for i in range(nested_lines_num - 1):
                start_line = process_num * i
                end_line = process_num * (i+1)
                futures = [legal_preprocessing_text.remote(lines[start_line:end_line][j]) for j in range(process_num)]
                results = ray.get(futures)

                if i == nested_lines_num - 2:
                    futures = [legal_preprocessing_text.remote(lines[end_line:][j]) for j in range(len(lines) - end_line)]
                    results = ray.get(futures)

                sentences = list(chain.from_iterable(results))
                sentence_list.append(sentences)

        sentence_list = list(chain.from_iterable(sentence_list))

        with open(post, 'a', encoding='utf-8') as fp:
            fp.write("\n".join(sentence_list))
            
        pbar.n += 1
        pbar.refresh()
        time.sleep(0.01)

    pbar.close() 

In [157]:
preprocessing_corpus_txt(pro_total_corpus_path_list, post_total_corpus_path_list)

[Size]
The number of preprocessing corpus: 35

[Order]


100%|██████████| 35/35 [03:55<00:00,  6.72s/it]


In [154]:
ray.shutdown()

In [ ]:
def merge_and_deduplicate_corpus_txt(preprocessing_corpus_path, merge_corpus_path,
                                      deduplicate_corpus_path):
    
    corpus_list = glob(preprocessing_corpus_path)
    corpus_list = sorted_list(corpus_list)
    
    with open(merge_corpus_path, 'w', encoding='utf-8') as f:
        for corpus in corpus_list:
            with open(corpus, encoding='utf-8') as text:
                for line in text:
                    f.write(line)
                    
    with open(deduplicate_corpus_path, 'w', encoding='utf-8') as f1:
        with open(merge_corpus_path, encoding='utf-8') as f2:
            lines = f2.read().splitlines()
            single_sentence_dict = dict.fromkeys(lines)
            single_sentence_list = list(single_sentence_dict)
            f1.write("\n".join(single_sentence_list))                

In [ ]:
preprocessing_corpus_path = "AIHUB_corpus/exploration/legal_regulations_(such_as_terms_and_conditions_of_judgment)_text_analysis_data_post/AIHUB_legal_regulations_(such_as_terms_and_conditions_of_judgment)_text_analysis_data_" +"*.txt"
merge_corpus_path = 'AIHUB_corpus/duplicate/AIHUB_legal_regulations_(such_as_terms_and_conditions_of_judgment)_text_analysis_data.txt'
deduplicate_corpus_path = 'AIHUB_corpus/AIHUB_legal_regulations_(such_as_terms_and_conditions_of_judgment)_text_analysis_data.txt'

In [ ]:
merge_and_deduplicate_corpus_txt(preprocessing_corpus_path, merge_corpus_path, 
                                  deduplicate_corpus_path)